In [ ]:
!pip install imutils

In [ ]:
#####Imports
import os
import imutils #!pip install imutils
import numpy as np 
import pandas as pd
from matplotlib import pyplot as plt
import cv2
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.layers import Flatten, Dense, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, History

# View and re-organize dataframe

In [ ]:
#viewing data
data = pd.read_csv('../input/pokemon-images-and-types/pokemon.csv')
print('Original data')
print(data.head(5))
#view and parse image folder
image_directory = "../input/pokemon-images-and-types/images/images/"
pokemon_filenames = os.listdir(image_directory)
pnames, primary_types = [], []
for path in os.listdir(image_directory):
        #isolating pokemon name
        pokemon_name = path.split('.')[0]
        #isolating pokemon primary type
        row = data[data['Name']==pokemon_name]
        type1 = data['Type1'][int(row.index.values)]
        #appending to new list
        pnames.append(pokemon_name.capitalize())
        primary_types.append(type1)      

In [ ]:
#Create new dataframe/data
pokemon_d = {'Filename':pokemon_filenames, 'Pokemon':pnames, 'Primary_Type':primary_types}
pokemon_data = pd.DataFrame(pokemon_d)
print('Pokemon dataframe shape (row,col):', pokemon_data.shape) #(row, col)
print(pokemon_data.head(5))

# Data prep

In [ ]:
#view and resize images
def preprocess(directory, n, plot = False, save = False, outdir = ''):
    '''
    Given a list of image paths, output n preprocessed images
    directory: folder that contain images
    n: number of images to preprocess
    plot: boolean to display images
    save: boolean to save preprocess images
    outdir: string to save preprocessed images
    '''
    for i in range(n):
        #image = cv2.imread(directory + i)
        image = cv2.imread(directory + pokemon_filenames[i], cv2.IMREAD_UNCHANGED) #change png transparent to white
        image = imutils.resize(image, width = 120) #orignal shape is 120
        (h, w) = image.shape[:2] #omitted channels
        if plot:
            plt.imshow(image)
            plt.title(f"{pnames[i]}: {primary_types[i]}")
            plt.show()
        try:
            if save:
                dest = os.path.join(outdir, pokemon_filenames[i])
                os.makedirs(os.path.dirname(dest), mode = 0o755, exist_ok = True)
                plt.imsave(dest, image)
        except Exception as e:
            print(e)
            print("Error reading image:", pokemon_filenames[i])
    return

preprocess(image_directory, 4, plot=True)

In [ ]:
#split using np: 60%, 20%, 20% split for training, validation and test sets
train, validation, test = np.split(pokemon_data, [int(.6*len(pokemon_data)), int(.8*len(pokemon_data))])
train, validation, test = train.reset_index(), validation.reset_index(), test.reset_index()
display(train, validation, test)

# Model

In [ ]:
#Designing model
model = VGG16(include_top=False, input_shape=(120,120,3))
flat = Flatten(input_shape=(3,3,512))(model.output)
dense = Dense(512, activation='relu')(flat)
output = Dense(18, activation='softmax')(dense)
#new model
model = Model(inputs=model.inputs, outputs=output)
model.summary()
#plot_model(model)

In [ ]:
#Compile model & prep/fit generators
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
datagen = ImageDataGenerator(rescale=1/255)

train_generator = datagen.flow_from_dataframe(
dataframe=train,
directory="../input/pokemon-images-and-types/images/images/",
x_col="Filename",
y_col="Primary_Type",
subset="training",
batch_size=50,
seed=0,
shuffle=True,
classes=['Water','Normal','Grass', 'Bug', 'Fire', 'Electric', 'Ground', 'Poison', 'Fairy', 'Fighting', 'Psychic', 'Rock', 'Ghost', 'Ice', 'Flying', 'Dragon', 'Dark', 'Steel'],
target_size=(120,120))

valid_generator = datagen.flow_from_dataframe(
dataframe=validation,
directory="../input/pokemon-images-and-types/images/images/",
x_col="Filename",
y_col="Primary_Type",
subset="training",
batch_size=50,
seed=0,
shuffle=True,
classes=['Water','Normal','Grass', 'Bug', 'Fire', 'Electric', 'Ground', 'Poison', 'Fairy', 'Fighting', 'Psychic', 'Rock', 'Ghost', 'Ice', 'Flying', 'Dragon', 'Dark', 'Steel'],
target_size=(120,120))

test_generator = datagen.flow_from_dataframe(
dataframe=test,
directory="../input/pokemon-images-and-types/images/images/",
x_col="Filename",
y_col="Primary_Type",
subset="training",
batch_size=50,
seed=0,
shuffle=True,
classes=['Water','Normal','Grass', 'Bug', 'Fire', 'Electric', 'Ground', 'Poison', 'Fairy', 'Fighting', 'Psychic', 'Rock', 'Ghost', 'Ice', 'Flying', 'Dragon', 'Dark', 'Steel'],
target_size=(120,120))

In [ ]:
#Fit model
#filepath = "../output/kaggle/working/VGG16_model.hdf5"
#checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=2, save_best_only=True, mode='max')
history = model.fit(train_generator,
                             steps_per_epoch = 10, #num of training images/batch_size
                             epochs = 5,
                             verbose = 2,
                             validation_data = valid_generator,
                             validation_steps= 10) #num of validation images/batch_size
                             #callbacks = [checkpoint]

# Performance

In [ ]:
results = model.evaluate_generator(test_generator, steps=20, verbose=1)
print("loss: ",results[0],"accuracy: ",results[1])